## Depencies

In [ ]:
%pip install numpy
%pip install matplotlib

In [ ]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from IPython.display import HTML

## Leg Mechanism

In [ ]:
class LegBase:
  def __init__(self, angle0_init=2*np.pi/3, angle5_init=np.pi/3):
    self.L0 = 8.16
    self.L1 = 8.0
    self.L2 = 10.0
    self.L3 = 8.0
    self.L4 = 13.0
    self.L5 = 8.0
    self.angle0 = angle0_init
    self.angle5 = angle5_init

## Kinematics Analysis in $R^2$

In [ ]:
class LegKinematics2D(LegBase):
  def __init__(self):
    super().__init__()

### Forward Kinematics

In [86]:
def calc_ang2pnt(self, angle0, angle5):
  '''
  Forward kinematics
  Given the angles of the motors, calculate the state of the leg, including the position of the points and the angles of the joints
  Return the points and angles of the joints
  '''
  p0 = np.array([0, 0])
  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p5 = np.array([self.L0, 0])
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5
  L14 = np.linalg.norm(p4-p1)

  if(L14 > self.L4 + self.L2):
    print("Unreachable")
    return None
  elif (L14 == self.L4 + self.L2):
    print("Singularity")

  angle_214 = acos((self.L2**2 + L14**2 - self.L4**2) / (2 * self.L2 * L14))
  rotation_matrix1 = np.array([[np.cos(angle_214), -np.sin(angle_214)],
                              [np.sin(angle_214), np.cos(angle_214)]])
  rotation_matrix2 = np.array([[np.cos(-angle_214), -np.sin(-angle_214)],
                              [np.sin(-angle_214), np.cos(-angle_214)]])
  
  unit_vector_14 = (p4 - p1) / L14
  p2_1 = p1 + self.L2 * rotation_matrix1 @ unit_vector_14
  p2_2 = p1 + self.L2 * rotation_matrix2 @ unit_vector_14
  p2 = p2_1 if p2_1[1] < p2_2[1] else p2_2

  # Translate p2 to p3
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  # Store the states
  points = np.array([p0, p1, p2, p3, p4, p5])

  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5
  ])

  return points, angles

LegKinematics2D.calc_ang2pnt = calc_ang2pnt

Visualizing the leg

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

# Initialize the points and lines
legkinematics = LegKinematics2D()
points = np.zeros((6, 2))  # 6 points in 2D
scat = ax.scatter(points[:, 0], points[:, 1], color='r')
line1, = ax.plot([], [], color='b') # Line p0 -> p1 -> p2 -> p4 -> p5
line2, = ax.plot([], [], color='b') # Line p2 -> p3

# Update function for animation
angle0 = np.linspace(np.pi, np.pi/2, 200)
angle5 = np.linspace(np.pi/2, 0, 200)

def update(frame):
  ang0 = angle0[frame]
  ang5 = angle5[frame]
  points[:6] = legkinematics.calc_ang2pnt(ang0, ang5)[0]

  # Update scatter plot
  scat.set_offsets(points)

  # Update lines
  line1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
  line2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])

  return scat, line1, line2

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_xlim([-15, 20])
ax.set_ylim([-30, 5])
ax.set_aspect('equal')

ani = FuncAnimation(fig, update, frames=200, interval=20, blit=True)
HTML(ani.to_jshtml())

### Inverse Kinematics

In [88]:
def calc_pnt2ang(self, x, y):
  '''
  Calculate the inverse kinematics using the geometric method.
  Given the position of the foot, calculate the angles of the motors,
  Return the angles of the motors, which are p0 and p5.
  '''

  # Calculate the angle of the motor 0
  L03 = np.linalg.norm(np.array([x, y]))
  angle_305 = atan2(-y, x)
  if angle_305 < 0:
    angle_305 += np.pi
  angle_301 = acos((self.L1**2 + L03**2 - (self.L2 + self.L3)**2) / (2 * self.L1 * L03))
  angle0 = angle_305 + angle_301

  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p2 = (np.array([x, y]) * self.L2 + p1 * self.L3) / (self.L2 + self.L3)
  p5 = np.array([self.L0, 0])

  # Calculate the angle of the motor 5
  L25 = np.linalg.norm(p2 - p5)
  angle_250 = atan2(-p2[1], self.L0 - p2[0])
  if angle_250 < 0:
    angle_250 += np.pi
  angle_254 = acos((L25**2 + self.L5**2 - self.L4**2) / (2 * L25 * self.L5))
  angle5 = np.pi - (angle_250 + angle_254)

  # Store the states
  p0 = np.array([0, 0])
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  points = np.array([p0, p1, p2, p3, p4, p5])

  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5
  ])

  return angles, points

LegKinematics2D.calc_pnt2ang = calc_pnt2ang


Check if inverse kinematics fits the forward kinematics

In [ ]:
legkinematics = LegKinematics2D()
angle0, angle5 = 2*np.pi/3, np.pi/3
print("Forward kinematics")
print("Given Angles: ", angle0, angle5)

points, angles = legkinematics.calc_ang2pnt(angle0, angle5)
print("Points: ", points)

print("\nInverse kinematics")
angles, points = legkinematics.calc_pnt2ang(points[3][0], points[3][1])
print("Given Point: ", points[3])
print("Angle0: ", angles[0], ", Angle5: ", angles[5])

### Differential Forward Kinematics

In [ ]:
def calc_omg2vel(self, omega0, omega5, angles):
  '''
  Calculate the velocity of the foot given the angular velocity of the motors
  '''
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])
  omega1 = omega0 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  vx2 = -self.L1 * omega0 * sin(angles[0]) - self.L2 * omega1 * sin(angles[1])
  vy2 = -self.L1 * omega0 * cos(angles[0]) - self.L2 * omega1 * cos(angles[1])

  vx3 = vx2 - self.L3 * omega1 * sin(angles[1])
  vy3 = vy2 - self.L3 * omega1 * cos(angles[1])
  
  return vx3, vy3

LegKinematics2D.calc_omg2vel = calc_omg2vel


Check if the result of the differential forward kinematics fit that of the forward kinematics. If it does, you will see only one vector in the left plot and two curves in the right plot, because the calculated values are the same as the actual values.

In [ ]:
class Verifier2D:
  def __init__(self):
    self.fig, (self.ax_mech, self.ax_vel) = plt.subplots(1, 2, figsize=(15, 5))

    # Set up axes
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_aspect('equal')
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')
    # self.ax_mech.legend()
    
    self.ax_vel.set_xlim(0, 2)  # Match your total_time
    self.ax_vel.set_ylim(-20, 20)
    self.ax_vel.grid(True)
    self.ax_vel.set_title('P3 Velocity Components')
    self.ax_vel.set_xlabel('Time (s)')
    self.ax_vel.set_ylabel('Velocity')
    # self.ax_vel.legend()

    # Data storage
    self.times = []
    self.actual_velocities = []
    self.calculated_velocities = []
    self.points_history = []

    # Initialize empty lines for the mechanism
    self.line1, = self.ax_mech.plot([], [], 'blue', lw=2, label='Chain 1')
    self.line2, = self.ax_mech.plot([], [], 'blue', lw=2, label='Chain 2')
    self.joints, = self.ax_mech.plot([], [], 'ro', label='Joints')
    self.calc_vector = self.ax_mech.quiver([], [], [], [], color='r', scale=1, units="xy", label='V Actual')
    self.actual_vector = self.ax_mech.quiver([], [], [], [], color='g', scale=1, units="xy", label='V Calc')

    # Initialize velocity plot lines
    self.vx_actual, = self.ax_vel.plot([], [], label='Vx')
    self.vy_actual, = self.ax_vel.plot([], [], label='Vy')
    self.vx_calc, = self.ax_vel.plot([], [], label='Vx Calc')
    self.vy_calc, = self.ax_vel.plot([], [], label='Vy Calc')
    
    self.ax_mech.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
    self.ax_vel.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

  def update(self, frame):
    points = self.points_history[frame]
    
    # Update chain
    self.line1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
    self.line2.set_data(points[[0, 5, 4, 2], 0], points[[0, 5, 4, 2], 1])
    self.joints.set_data(points[:, 0], points[:, 1])

    # Update velocity vector at p3
    if frame > 0:
      actual_vel = self.actual_velocities[frame]
      calc_vel = self.calculated_velocities[frame]

      # Plot velocity vector
      self.calc_vector.set_offsets(points[3])
      self.calc_vector.set_UVC(calc_vel[0], calc_vel[1])
      self.actual_vector.set_offsets(points[3])
      self.actual_vector.set_UVC(actual_vel[0], actual_vel[1])
        
    # Plot velocity components over time
    time = self.times[:frame+1]
    actual_vx = [v[0] for v in self.actual_velocities[:frame+1]]
    actual_vy = [v[1] for v in self.actual_velocities[:frame+1]]
    calc_vx = [v[0] for v in self.calculated_velocities[:frame+1]]
    calc_vy = [v[1] for v in self.calculated_velocities[:frame+1]]

    self.vx_actual.set_data(time, actual_vx)
    self.vy_actual.set_data(time, actual_vy)
    self.vx_calc.set_data(time, calc_vx)
    self.vy_calc.set_data(time, calc_vy)

    return self.line1, self.line2, self.joints
      
  def animate(self, points_history, times, actual_velocities, calculated_velocities):
    self.points_history = points_history
    self.times = times
    self.actual_velocities = actual_velocities
    self.calculated_velocities = calculated_velocities
    
    anim = FuncAnimation(self.fig, self.update,
                        frames=len(times), 
                        interval=50,
                        blit=True)
    
    plt.tight_layout()
    return HTML(anim.to_jshtml())

In [ ]:
# Initialize the verifier
verifier = Verifier2D()
points_history = []
times = []
actual_velocities = []
calc_velocities = []

# Time variables
time = 0
dt = 0.05
total_time = 1

# Initial points
angle0, angle5 = np.pi, np.pi/2
omega0, omega5 = -1, -1
prev_points, _ = legkinematics.calc_ang2pnt(angle0, angle5)

while time < total_time:
  # Update the angles
  angle0 = (angle0 + omega0 * dt) % (2*np.pi)
  angle5 = (angle5 + omega5 * dt) % (2*np.pi)

  # Forwards kinematics
  points, angles = legkinematics.calc_ang2pnt(angle0, angle5)

  calc_p3_velocity = legkinematics.calc_omg2vel(omega0, omega5, angles)
  actual_p3_velocity = (points[3] - prev_points[3]) / dt

  # Store data
  points_history.append(points)
  times.append(time)
  actual_velocities.append(actual_p3_velocity)
  calc_velocities.append(calc_p3_velocity)

  prev_points = points
  time += dt

animation = verifier.animate(points_history, times, actual_velocities, calc_velocities)
display(animation)

### Differential Inverse Kinematics

In [ ]:
def calc_vel2omg(self, vx3, vy3, point3):
  '''
  Calculate the angular velocity of the motors given the velocity and position of the foot
  '''
  angles, points = self.calc_pnt2ang(point3[0], point3[1])
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])

  i = -self.L1 * sin(angles[0]) - (self.L2 + self.L3) * sin(angles[1]) * (a*h-e*d) / (c*h-d*g)
  j = -sin(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g) 
  k = -self.L1 * cos(angles[0]) - (self.L2 + self.L3) * cos(angles[1]) * (a*h-e*d) / (c*h-d*g)
  l = -cos(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g)

  omega0 = (l * vx3 - j * vy3) / (i * l - j * k)
  omega5 = (-k * vx3 + i * vy3) / (i * l - j * k)

  return omega0, omega5

LegKinematics2D.calc_vel2omg = calc_vel2omg


Check if omg2vel and vel2omg are consistent

In [ ]:
legkinematics = LegKinematics2D()

# Given omegas and angles
omg0, omg5 = -0.5, -1
angle0, angle5 = 2*np.pi/3, np.pi/3

print("\nVelocity calculation")
print("Given Omega0: ", omg0, ", Omega5: ", omg5)
print("Given Angle0: ", angle0, ", Angle5: ", angle5)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5)
vx, vy = legkinematics.calc_omg2vel(omg0, omg5, angles)
print("Velocity: ", vx, vy)

print("\nAngular velocity calculation")
print("Given Velocity: ", vx, vy)
print("Given Point: ", points[3])
angle0, angle5 = legkinematics.calc_pnt2ang(points[3][0], points[3][1])
omega0, omega5 = legkinematics.calc_vel2omg(vx, vy, points[3])
print("Omega0: ", omega0, ", Omega5: ", omega5)

## Kinematics Analysis in $R^3$

Now, we will work on the 3D leg model, expanding the previous analysis to general 3D space.

In [ ]:
class LegKinematics(LegBase):
  def __init__(self):
    super().__init__()

### Forward Kinematics

In [ ]:
def calc_ang2pnt(self, angle0, angle5, angle6):
  '''
  Forward kinematics
  Given the angles of the motors, calculate the state of the leg, including the position of the points and the angles of the joints
  Return the points and angles of the joints
  '''
  # Start from 2D kinematics
  p0 = np.array([0, 0])
  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p5 = np.array([self.L0, 0])
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5
  L14 = np.linalg.norm(p4-p1)

  if(L14 > self.L4 + self.L2):
    print("Unreachable")
    return None
  elif (L14 == self.L4 + self.L2):
    print("Singularity")

  angle_214 = acos((self.L2**2 + L14**2 - self.L4**2) / (2 * self.L2 * L14))
  rotation_matrix1 = np.array([[np.cos(angle_214), -np.sin(angle_214)],
                              [np.sin(angle_214), np.cos(angle_214)]])
  rotation_matrix2 = np.array([[np.cos(-angle_214), -np.sin(-angle_214)],
                              [np.sin(-angle_214), np.cos(-angle_214)]])
  
  unit_vector_14 = (p4 - p1) / L14
  p2_1 = p1 + self.L2 * rotation_matrix1 @ unit_vector_14
  p2_2 = p1 + self.L2 * rotation_matrix2 @ unit_vector_14
  p2 = p2_1 if p2_1[1] < p2_2[1] else p2_2

  # Translate p2 to p3
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  # Store the 2D states
  points = np.array([p0, p1, p2, p3, p4, p5])

  # Translate points from 2D to 3D
  offset_angle = - angle6 # Rotate the current plane to the x-y plane
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(offset_angle), -sin(offset_angle)],
    [0, sin(offset_angle), cos(offset_angle)]
  ])
  points = np.concatenate([points, np.zeros((6, 1))], axis=1)
  points = points @ transformation_matrix

  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5,
    angle6
  ])

  return points, angles

LegKinematics.calc_ang2pnt = calc_ang2pnt

Verify if the forward kinematics works.

In [ ]:
legkinematics = LegKinematics()
N = 100  # Number of frames

# Define oscillating angles
angles6 = np.linspace(0, np.pi/2, N//2)
angles6 = np.concatenate([angles6, angles6[::-1]])  # Oscillate
angles0 = np.linspace(np.pi, np.pi/2, N)
angles5 = np.linspace(np.pi/2, 0, N)

# Create figure with two subplots
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# Set limits and labels for both subplots
for ax in [ax1, ax2]:
    ax.set_xlim3d([-15, 20])
    ax.set_ylim3d([-30, 5])
    ax.set_zlim3d([-15, 15])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

# Set different viewpoints
ax1.view_init(vertical_axis='y', azim=90)
ax2.view_init(vertical_axis='y', azim=30)

# Initialize empty lines for both subplots
line1_1, = ax1.plot([], [], [], 'blue', markersize=5)
line1_2, = ax1.plot([], [], [], 'blue', markersize=5)
scatter1 = ax1.scatter([], [], [], color='red', s=50, label='Joints')

line2_1, = ax2.plot([], [], [], 'blue', markersize=5)
line2_2, = ax2.plot([], [], [], 'blue', markersize=5)
scatter2 = ax2.scatter([], [], [], color='red', s=50, label='Joints')

# Animation update function
def update(frame):
    angle0 = angles0[frame]
    angle5 = angles5[frame]
    angle6 = angles6[frame]

    points, _ = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
    if points is not None:
        # Update first subplot (Side view)
        line1_1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
        line1_2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])
        line1_1.set_3d_properties(points[[0, 1, 2, 3], 2])
        line1_2.set_3d_properties(points[[0, 1, 2, 4, 5], 2])
        scatter1._offsets3d = (points[:, 0], points[:, 1], points[:, 2])

        # Update second subplot (Front view)
        line2_1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
        line2_2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])
        line2_1.set_3d_properties(points[[0, 1, 2, 3], 2])
        line2_2.set_3d_properties(points[[0, 1, 2, 4, 5], 2])
        scatter2._offsets3d = (points[:, 0], points[:, 1], points[:, 2])

    return line1_1, line1_2, scatter1, line2_1, line2_2, scatter2

# Run animation
ani = FuncAnimation(fig, update, frames=N, interval=50)
HTML(ani.to_jshtml())

### Inverse Kinematics

In [98]:
def calc_pnt2ang(self, x, y, z):
  '''
  Calculate the inverse kinematics using the geometric method.
  Given the position of the foot, calculate the angles of the motors,
  Return the angles of the motors, which are p0 and p5.
  '''

  angle6 = atan2(-y, z) - np.pi/2
  if angle6 < 0:
    angle6 += np.pi

  # Translate the points from 3D to 2D
  offset_angle = angle6
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(offset_angle), -sin(offset_angle)],
    [0, sin(offset_angle), cos(offset_angle)]
  ])
  [x, y, z] = np.array([x, y, z]) @ transformation_matrix

  # Calculate the angle of the motor 0
  L03 = np.linalg.norm(np.array([x, y]))
  angle_305 = atan2(-y, x)
  if angle_305 < 0:
    angle_305 += np.pi
  angle_301 = acos((self.L1**2 + L03**2 - (self.L2 + self.L3)**2) / (2 * self.L1 * L03))
  angle0 = angle_305 + angle_301

  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p2 = (np.array([x, y]) * self.L2 + p1 * self.L3) / (self.L2 + self.L3)
  p5 = np.array([self.L0, 0])

  # Calculate the angle of the motor 5
  L25 = np.linalg.norm(p2 - p5)
  angle_250 = atan2(-p2[1], self.L0 - p2[0])
  if angle_250 < 0:
    angle_250 += np.pi
  angle_254 = acos((L25**2 + self.L5**2 - self.L4**2) / (2 * L25 * self.L5))
  angle5 = np.pi - (angle_250 + angle_254)

  # Store the states
  p0 = np.array([0, 0])
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  points = np.array([p0, p1, p2, p3, p4, p5])
  points = np.concatenate([points, np.zeros((6, 1))], axis=1)
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(-offset_angle), -sin(-offset_angle)],
    [0, sin(-offset_angle), cos(-offset_angle)]
  ])
  points = points @ transformation_matrix
  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5,
    angle6
  ])

  return angles, points

LegKinematics.calc_pnt2ang = calc_pnt2ang


Verify if the inverse kinematics consistent with the forward kinematics.

In [ ]:
# Given angles
angle0, angle5, angle6 = np.pi, np.pi/2, np.pi/4
print("Forward kinematics")
print("Given Angles: ", angle0, angle5, angle6)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
print("Points: ", points[3])

print("\nInverse kinematics")
print("Given Point: ", points[3])
angles, points = legkinematics.calc_pnt2ang(points[3][0], points[3][1], points[3][2])
print("Angles: ", angles[0], angles[5], angles[6])

### Differential Forward kinematics

In [23]:
def calc_omg2vel(self, omega0, omega5, omega6, angles, points):
  '''
  Calculate the velocity of the foot given the angular velocity of the motors
  '''
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])
  omega1 = omega0 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  vx2_2d = -self.L1 * omega0 * sin(angles[0]) - self.L2 * omega1 * sin(angles[1])
  vy2_2d = -self.L1 * omega0 * cos(angles[0]) - self.L2 * omega1 * cos(angles[1])

  vx3_2d = vx2_2d - self.L3 * omega1 * sin(angles[1])
  vy3_2d = vy2_2d - self.L3 * omega1 * cos(angles[1])

  vx3 = vx3_2d
  vy3 = vy3_2d * cos(angles[6]) + omega6 * np.sqrt(points[3][1]**2 + points[3][2]**2) * sin(angles[6])
  vz3 = vy3_2d * sin(angles[6]) - omega6 * np.sqrt(points[3][1]**2 + points[3][2]**2) * cos(angles[6])
  
  return vx3, vy3, vz3

LegKinematics.calc_omg2vel = calc_omg2vel


Verify if the differential forward kinematics consistent with the forward kinematics.

In [24]:
class Verifier:
  def __init__(self):
    # Create figure with two subplots
    self.fig = plt.figure(figsize=(15, 5))
        
    # Create 3D subplot for mechanism
    self.ax_mech = self.fig.add_subplot(121, projection='3d')
    
    # Set up axes
    self.ax_mech.set_xlabel("X")
    self.ax_mech.set_ylabel("Y")
    self.ax_mech.set_zlabel("Z")
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_zlim(-10, 10)
    self.ax_mech.set_box_aspect([35, 35, 20])
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')

    # Create 2D subplot for velocity
    self.ax_vel = self.fig.add_subplot(122)
    self.ax_vel.set_xlim(0, 2)
    self.ax_vel.set_ylim(-20, 20)
    self.ax_vel.grid(True)
    self.ax_vel.set_title('P3 Velocity Components')
    self.ax_vel.set_xlabel('Time (s)')
    self.ax_vel.set_ylabel('Velocity')

    # Data storage
    self.times = []
    self.actual_velocities = []
    self.calculated_velocities = []
    self.points_history = []

    # Dummy line for animation update
    self.dummyLine, = self.ax_mech.plot([], [], [])

    # Initialize velocity plot lines
    self.vx_actual, = self.ax_vel.plot([], [], label='Vx')
    self.vy_actual, = self.ax_vel.plot([], [], label='Vy')
    self.vz_actual, = self.ax_vel.plot([], [], label='Vz')
    self.vx_calc, = self.ax_vel.plot([], [], label='Vx Calc')
    self.vy_calc, = self.ax_vel.plot([], [], label='Vy Calc')
    self.vz_calc, = self.ax_vel.plot([], [], label='Vz Calc')
    
    self.ax_mech.view_init(vertical_axis='y', azim=30)
    self.ax_vel.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

  def update(self, frame):
    points = self.points_history[frame]

    self.ax_mech.cla()
    self.ax_mech.set_xlabel("X")
    self.ax_mech.set_ylabel("Y")
    self.ax_mech.set_zlabel("Z")
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_zlim(-10, 10)
    self.ax_mech.set_box_aspect([35, 35, 20])
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')
    
    # Update chain
    self.ax_mech.plot3D(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1], points[[0, 1, 2, 3], 2], 'blue')
    self.ax_mech.plot3D(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1], points[[0, 1, 2, 4, 5], 2], 'blue')
    self.ax_mech.scatter3D(points[:, 0], points[:, 1], points[:, 2], c='r', marker='o')
    
    # Update velocity vector at p3
    if frame > 0:
      actual_vel = self.actual_velocities[frame]
      calc_vel = self.calculated_velocities[frame]

      # Plot velocity vector
      self.ax_mech.quiver(points[3, 0], points[3, 1], points[3, 2], actual_vel[0], actual_vel[1], actual_vel[2], color='g', label='V Actual')
      self.ax_mech.quiver(points[3, 0], points[3, 1], points[3, 2], calc_vel[0], calc_vel[1], calc_vel[2], color='r', label='V Calc')
        
    # Plot velocity components over time
    time = self.times[:frame+1]
    actual_vx = [v[0] for v in self.actual_velocities[:frame+1]]
    actual_vy = [v[1] for v in self.actual_velocities[:frame+1]]
    actual_vz = [v[2] for v in self.actual_velocities[:frame+1]]
    calc_vx = [v[0] for v in self.calculated_velocities[:frame+1]]
    calc_vy = [v[1] for v in self.calculated_velocities[:frame+1]]
    calc_vz = [v[2] for v in self.calculated_velocities[:frame+1]]

    self.vx_actual.set_data(time, actual_vx)
    self.vy_actual.set_data(time, actual_vy)
    self.vz_actual.set_data(time, actual_vz)
    self.vx_calc.set_data(time, calc_vx)
    self.vy_calc.set_data(time, calc_vy)
    self.vz_calc.set_data(time, calc_vz)

    return self.dummyLine,
      
  def animate(self, points_history, times, actual_velocities, calculated_velocities):
    self.points_history = points_history
    self.times = times
    self.actual_velocities = actual_velocities
    self.calculated_velocities = calculated_velocities
    
    anim = FuncAnimation(self.fig, self.update,
                        frames=len(times), 
                        interval=50,
                        blit=True)
    
    plt.tight_layout()
    return HTML(anim.to_jshtml())

In [ ]:
# Initialize the verifier
verifier = Verifier()
points_history = []
times = []
actual_velocities = []
calc_velocities = []

# Time variables
time = 0
dt = 0.02
total_time = 1

# Initial points
angle0, angle5, angle6 = np.pi, np.pi/2, 0
omega0, omega5, omega6 = -1, -0.5, 0.5
prev_points, _ = legkinematics.calc_ang2pnt(angle0, angle5, angle6)

while time < total_time:
  # Update the angles
  angle0 = (angle0 + omega0 * dt) % (2*np.pi)
  angle5 = (angle5 + omega5 * dt) % (2*np.pi)
  angle6 = (angle6 + omega6 * dt) % (2*np.pi)

  # Forwards kinematics
  points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
  calc_p3_velocity = legkinematics.calc_omg2vel(omega0, omega5, omega6, angles, points)
  actual_p3_velocity = (points[3] - prev_points[3]) / dt

  # Store data
  points_history.append(points)
  times.append(time)
  actual_velocities.append(actual_p3_velocity)
  calc_velocities.append(calc_p3_velocity)

  prev_points = points
  time += dt

animation = verifier.animate(points_history, times, actual_velocities, calc_velocities)
display(animation)

### Differential Inverse kinematics

In [26]:
def calc_vel2omg(self, vx3, vy3, vz3, points, angles):
  '''
  Calculate the angular velocity of the motors given the velocity and position of the foot
  '''
  # Convert the velocity to 2D
  vx3_2d = vx3
  vy3_2d = vy3 * cos(angles[6]) + vz3 * sin(angles[6])
  v_omega6 = vy3 * sin(angles[6]) - vz3 * cos(angles[6])
  omega6 = v_omega6 / np.sqrt(points[3][1]**2 + points[3][2]**2)
  
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])

  i = -self.L1 * sin(angles[0]) - (self.L2 + self.L3) * sin(angles[1]) * (a*h-e*d) / (c*h-d*g)
  j = -sin(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g) 
  k = -self.L1 * cos(angles[0]) - (self.L2 + self.L3) * cos(angles[1]) * (a*h-e*d) / (c*h-d*g)
  l = -cos(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g)

  omega0 = (l * vx3_2d - j * vy3_2d) / (i * l - j * k)
  omega5 = (-k * vx3_2d + i * vy3_2d) / (i * l - j * k)

  return omega0, omega5, omega6

LegKinematics.calc_vel2omg = calc_vel2omg


Verify if the differential inverse kinematics consistent with the differential forward kinematics.

In [ ]:
omega0, omega5, omega6 = -1, -1, 1
angle0, angle5, angle6 = np.pi, np.pi/2, np.pi/4
print("\nVelocity calculation")
print("Given Omega0: ", omega0, ", Omega5: ", omega5, ", Omega6: ", omega6)
print("Given Angle0: ", angle0, ", Angle5: ", angle5, ", Angle6: ", angle6)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
vx, vy, vz = legkinematics.calc_omg2vel(omega0, omega5, omega6, angles, points)
print("End Point: ", points[3])
print("Velocity: ", vx, vy, vz)

print("\nAngular velocity calculation")
print("Given Velocity: ", vx, vy, vz)
print("Given Point: ", points[3])
angles, points = legkinematics.calc_pnt2ang(points[3][0], points[3][1], points[3][2])
omega0, omega5, omega6 = legkinematics.calc_vel2omg(vx, vy, vz, points, angles)
print("Angle0: ", angles[0], ", Angle5: ", angles[5], ", Angle6: ", angles[6])
print("Omega0: ", omega0, ", Omega5: ", omega5, ", Omega6: ", omega6)

## DH method

In this section, we will redo the kinematics analysis using the DH method and the Jacobian matrix.

### Denavit-Hartenberg(DH) method

$ a_i $: distance between $ O_{i-1} $ and $ O_i $ measured along $ x_{i-1} $  
$ d_i $: distance between $ O_{i-1} $ and $ O_i $ measured along $ z_{i-1} $  
$ \alpha_i $: angle between $ z_{i-1} $ and $ z_i $ measured about $ x_i $ when the rotation is made counterclockwise.  
$ \theta_i $: angle between $ x_{i-1} $ and $ x_i $ measured about $ z_{i-1} $ when the rotation is made counterclockwise.  

### Reference

[YT video - 2.13 D-H parameters of 4-bar parallel robot](https://www.youtube.com/watch?v=oIz5L0VpLQo&t=332s)  
[YT video - Introduction to DH Convention](https://www.youtube.com/watch?v=JSEPoyIypHQ)

### Transformation matrix

$$
T = \begin{bmatrix}
\cos(\theta) & -\sin(\theta)\cos(\alpha) & \sin(\theta)\sin(\alpha)  & a\cos(\theta) \\
\sin(\theta) & \cos(\theta)\cos(\alpha) & -\cos(\theta)\sin(\alpha) & a\sin(\theta) \\
0 & \sin(\alpha) & \cos(\alpha) & d \\
0 & 0 & 0 & 1
\end{bmatrix}

 = \begin{bmatrix}
R_{11} & R_{12} & R_{13} & p_x \\
R_{21} & R_{22} & R_{23} & p_y \\
R_{31} & R_{32} & R_{33} & p_z \\
0 & 0 & 0 & 1
\end{bmatrix}

= 
\begin{bmatrix}
  R_{3 \times 3} & p_{3 \times 1} \\
  0 & 1
\end{bmatrix}
$$

In [28]:
def get_coordinate_transform(self, theta, d, a, alpha):
  return np.array([[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
                   [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
                   [0, np.sin(alpha), np.cos(alpha), d],
                   [0, 0, 0, 1]])

LegKinematics.get_coordinate_transform = get_coordinate_transform

In [29]:
def get_transform_matrix(self, angles):
  # Base to joint0
  Ab0_1 = self.get_coordinate_transform(theta=-np.pi/2, d=0, a=0,  alpha=0)
  Ab0_2 = self.get_coordinate_transform(theta=0, d=0, a=0,  alpha=-np.pi/2)
  Ab0_3 = self.get_coordinate_transform(theta=angles[0], d=0, a=0,  alpha=0)
 
  # Joint0 to joint1
  A01_1 = self.get_coordinate_transform(theta=0, d=0, a=0, alpha=np.pi/2)
  A01_2 = self.get_coordinate_transform(theta=np.pi/2-angles[1], d=0, a=0, alpha=0)

  # Joint 1 to joint 2
  A12_1 = self.get_coordinate_transform(theta=0, d=0, a=self.L1, alpha=0)
  A12_2 = self.get_coordinate_transform(theta=angles[1]-angles[2], d=0, a=0, alpha=0)

  # Joint 2 to end effector
  A2e = self.get_coordinate_transform(theta=0, d=0, a=self.L2+self.L3,  alpha=0)

  T0 = Ab0_1 @ Ab0_2 @ Ab0_3
  T1 = T0 @ A01_1 @ A01_2
  T2 = T1 @ A12_1 @ A12_2
  Te = T2 @ A2e

  T = [T0, T1, T2, Te]
  return T

LegKinematics.get_transform_matrix = get_transform_matrix


In [ ]:
angle0, angle5, angle6 = np.pi, np.pi/2, 0
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
angles = np.array([angles[6], angles[0], angles[1]])
T = legkinematics.get_transform_matrix(angles)

base = np.array([0, 0, 0, 1])
joint0 = T[0] @ base
joint1 = T[1] @ base
joint2 = T[2] @ base
end_effector = T[3] @ base

# Extract rotation matrix
R0 = T[0][:3, :3]
R1 = T[1][:3, :3]
R2 = T[2][:3, :3]
Re = T[3][:3, :3]

print("\nX axis direction of each frame in base frame: ")
x = np.array([1, 0, 0, 1])
print("Base: ", x)
print("Joint0: ", T[0] @ x)
print("Joint1: ", T[1] @ x)
print("Joint2: ", T[2] @ x)
print("End effector: ", T[3] @ x)


Verify if the transformation matrix fits the forward kinematics.

In [ ]:
angle0, angle5, angle6 = np.pi, np.pi/2, 0
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
# print("Points: ")
# for p in points:
  # print(p)

angles = np.array([angles[6], angles[0], angles[1]])
T = legkinematics.get_transform_matrix(angles)
print("\nTransform Matrix")
for t in T:
  print(t[:3, 3])

### Jacobian matrix

$$
v = J \times \dot{q}
$$

$$
\dot{q} = J^{-1} \times v
$$

$$
J = \begin{bmatrix}
J_{pi} \\
J_{oi}
\end{bmatrix}

= \begin{bmatrix}
z_{i-1} \times (p_n - p_{i-1}) \\
z_{i-1}
\end{bmatrix}
$$
for revolute joint

In [32]:
def get_jacobian(self, T_list):
        T0, T1, T2, Te = T_list

        # Extract positions (translation vectors)
        p = [T[:3, 3] for T in T_list]  # p0, p1, p2, p3, p4, p5, pe
        pe = p[-1]  # End-effector position

        # Extract rotation matrices and z-axes
        R = [T[:3, :3] for T in T_list]
        z = [R_i[:, 2] for R_i in R]  # z-axis is third column (DH convention)

        # Active joints: 0 (theta0), 1 (theta1), 5 (theta5)
        active_joints = [0, 1, 2]  # Indices of motor-driven joints
        J = np.zeros((6, 3))  # 6 rows (vx, vy, vz, wx, wy, wz), 3 columns

        # Compute Jacobian columns
        for i, joint_idx in enumerate(active_joints):
            z_i = z[joint_idx]  # Rotation axis in base frame
            p_i = p[joint_idx]  # Position of joint frame
            J[0:3, i] = np.cross(z_i, pe - p_i)  # Linear velocity contribution
            J[3:6, i] = z_i  # Angular velocity contribution

        return J

LegKinematics.get_jacobian = get_jacobian

In [33]:
def calc_omega1(self, omega0, omega5, angles):
  '''
  Calculate the angular velocity of the motor 1 given the angular velocities of the motors 0 and 5
  '''
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])
  omega1 = omega0 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  return omega1

LegKinematics.calc_omega1 = calc_omega1

Verify if the Jacobian matrix fits the differential forward kinematics.

In [ ]:
angle0, angle5, angle6 = np.pi, np.pi/2, 0
omega0, omega5, omega6 = 1, 1, 0
print("Given Angles: ", angle0, angle5, angle6)
print("Given Omegas: ", omega0, omega5, omega6)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
vx, vy, vz = legkinematics.calc_omg2vel(omega0, omega5, omega6, angles, points)
print("\nEnd Point: ", points[3])
print("Velocity: ", vx, vy, vz)

T = legkinematics.get_transform_matrix(np.array([angle6, angle0, angles[1]]))
print("\nT[4] position:", T[3][0:3, 3])
J = legkinematics.get_jacobian(T)
omega1 = legkinematics.calc_omega1(omega0, omega5, angles)
vel = J @ np.array([omega6, omega0, omega1])
print("Vel from Jacobian: ", vel)
